<a href="https://colab.research.google.com/github/keethu12345/NLP-Transformers/blob/main/Creating_Own_Dataset_HF_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To download the issues of the githib we use the requests library of Python

In [ ]:
!pip install requests

To download the issues of the githib we use the requests library of Python

In [ ]:
# To make get reuquests invoke requests.get() method\
import requests

url = "https://api.github.com/repos/huggingface/datasets/issues?page=1&per_page=1"
response = requests.get(url)

In [ ]:
response.status_code#contains request info

200

In [ ]:
response.json()# generating the payload

[{'url': 'https://api.github.com/repos/huggingface/datasets/issues/6976',
  'repository_url': 'https://api.github.com/repos/huggingface/datasets',
  'labels_url': 'https://api.github.com/repos/huggingface/datasets/issues/6976/labels{/name}',
  'comments_url': 'https://api.github.com/repos/huggingface/datasets/issues/6976/comments',
  'events_url': 'https://api.github.com/repos/huggingface/datasets/issues/6976/events',
  'html_url': 'https://github.com/huggingface/datasets/pull/6976',
  'id': 2357107203,
  'node_id': 'PR_kwDODunzps5yrmNP',
  'number': 6976,
  'title': 'Ensure compatibility with numpy 2.0.0',
  'user': {'login': 'KennethEnevoldsen',
   'id': 23721977,
   'node_id': 'MDQ6VXNlcjIzNzIxOTc3',
   'avatar_url': 'https://avatars.githubusercontent.com/u/23721977?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/KennethEnevoldsen',
   'html_url': 'https://github.com/KennethEnevoldsen',
   'followers_url': 'https://api.github.com/users/KennethEnevoldsen/followers

In [ ]:
GITHUB_TOKEN = "your_own_token"
headers = {"Authorization": f"token {GITHUB_TOKEN}"}

Function to download all the issues of the given repository!

In [ ]:
import time
import math
from pathlib import Path
import pandas as pd
from tqdm.notebook import tqdm


def fetch_issues(
    owner="huggingface",
    repo="datasets",
    num_issues=10_000,
    rate_limit=10_000,
    issues_path=Path("."),
):
    if not issues_path.is_dir():
        issues_path.mkdir(exist_ok=True)

    batch = []
    all_issues = []
    per_page = 100  # Number of issues to return per page
    num_pages = math.ceil(num_issues / per_page)
    base_url = "https://api.github.com/repos"

    for page in tqdm(range(num_pages)):
        # Query with state=all to get both open and closed issues
        query = f"issues?page={page}&per_page={per_page}&state=all"
        issues = requests.get(f"{base_url}/{owner}/{repo}/{query}", headers=headers)
        batch.extend(issues.json())

        if len(batch) > rate_limit and len(all_issues) < num_issues:
            all_issues.extend(batch)
            batch = []  # Flush batch for next time period
            print(f"Reached GitHub rate limit. Sleeping for one hour ...")
            time.sleep(60 * 60 + 1)

    all_issues.extend(batch)
    df = pd.DataFrame.from_records(all_issues)
    df.to_json(f"{issues_path}/{repo}-issues.jsonl", orient="records", lines=True)
    print(
        f"Downloaded all the issues for {repo}! Dataset stored at {issues_path}/{repo}-issues.jsonl"
    )

In [ ]:
# Depending on your internet connection, this can take several minutes to run...
fetch_issues()

  0%|          | 0/100 [00:00<?, ?it/s]

Downloaded all the issues for datasets! Dataset stored at ./datasets-issues.jsonl


In [ ]:
!pip install datasets

In [ ]:
import pandas as pd
from datasets import Dataset
df = pd.read_json('datasets-issues.jsonl', orient='records', lines=True)
issues_dataset = Dataset.from_pandas(df, split="train")
issues_dataset

Dataset({
    features: ['url', 'repository_url', 'labels_url', 'comments_url', 'events_url', 'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels', 'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments', 'created_at', 'updated_at', 'closed_at', 'author_association', 'active_lock_reason', 'body', 'reactions', 'timeline_url', 'performed_via_github_app', 'state_reason', 'draft', 'pull_request'],
    num_rows: 6936
})

Cleaning up the data!

In [ ]:
sample = issues_dataset.shuffle(seed=666).select(range(10))

# Print out the URL and pull request entries
for url, pr in zip(sample["html_url"], sample["pull_request"]):
    print(f">> URL: {url}")
    print(f">> Pull request: {pr}\n")

>> URL: https://github.com/huggingface/datasets/pull/2260
>> Pull request: {'diff_url': 'https://github.com/huggingface/datasets/pull/2260.diff', 'html_url': 'https://github.com/huggingface/datasets/pull/2260', 'merged_at': '2021-05-07T08:36:17Z', 'patch_url': 'https://github.com/huggingface/datasets/pull/2260.patch', 'url': 'https://api.github.com/repos/huggingface/datasets/pulls/2260'}

>> URL: https://github.com/huggingface/datasets/pull/818
>> Pull request: {'diff_url': 'https://github.com/huggingface/datasets/pull/818.diff', 'html_url': 'https://github.com/huggingface/datasets/pull/818', 'merged_at': '2020-11-10T09:07:01Z', 'patch_url': 'https://github.com/huggingface/datasets/pull/818.patch', 'url': 'https://api.github.com/repos/huggingface/datasets/pulls/818'}

>> URL: https://github.com/huggingface/datasets/issues/5922
>> Pull request: None

>> URL: https://github.com/huggingface/datasets/issues/3977
>> Pull request: None

>> URL: https://github.com/huggingface/datasets/issues/

In [ ]:
issues_dataset = issues_dataset.map(
    lambda x: {"is_pull_request": False if x["pull_request"] is None else True}
)

Map:   0%|          | 0/6936 [00:00<?, ? examples/s]

Augmenting the dataset!

In [ ]:
import requests
issue_number = 2792
url = f"https://api.github.com/repos/huggingface/datasets/issues/{issue_number}/comments"
response = requests.get(url, headers=headers)
response.json()

[{'url': 'https://api.github.com/repos/huggingface/datasets/issues/comments/897594128',
  'html_url': 'https://github.com/huggingface/datasets/pull/2792#issuecomment-897594128',
  'issue_url': 'https://api.github.com/repos/huggingface/datasets/issues/2792',
  'id': 897594128,
  'node_id': 'IC_kwDODunzps41gDMQ',
  'user': {'login': 'bhavitvyamalik',
   'id': 19718818,
   'node_id': 'MDQ6VXNlcjE5NzE4ODE4',
   'avatar_url': 'https://avatars.githubusercontent.com/u/19718818?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/bhavitvyamalik',
   'html_url': 'https://github.com/bhavitvyamalik',
   'followers_url': 'https://api.github.com/users/bhavitvyamalik/followers',
   'following_url': 'https://api.github.com/users/bhavitvyamalik/following{/other_user}',
   'gists_url': 'https://api.github.com/users/bhavitvyamalik/gists{/gist_id}',
   'starred_url': 'https://api.github.com/users/bhavitvyamalik/starred{/owner}{/repo}',
   'subscriptions_url': 'https://api.github.com/users/

Let’s write a simple function that returns all the comments associated with an issue by picking out the body contents for each element in response.json()!

In [ ]:
def get_comments(issue_number):
    url = f"https://api.github.com/repos/huggingface/datasets/issues/{issue_number}/comments"
    response = requests.get(url, headers=headers)
    return [r["body"] for r in response.json()]


# Test our function works as expected
get_comments(2792)

["@albertvillanova my tests are failing here:\r\n```\r\ndataset_name = 'gooaq'\r\n\r\n    def test_load_dataset(self, dataset_name):\r\n        configs = self.dataset_tester.load_all_configs(dataset_name, is_local=True)[:1]\r\n>       self.dataset_tester.check_load_dataset(dataset_name, configs, is_local=True, use_local_dummy_data=True)\r\n\r\ntests/test_dataset_common.py:234: \r\n_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ \r\ntests/test_dataset_common.py:187: in check_load_dataset\r\n    self.parent.assertTrue(len(dataset[split]) > 0)\r\nE   AssertionError: False is not true\r\n```\r\nWhen I try loading dataset on local machine it works fine. Any suggestions on how can I avoid this error?",
 'Thanks for the help, @albertvillanova! All tests are passing now.']

Dataset.map() to add a new comments column to each issue in our dataset.

In [ ]:
# Depending on your internet connection, this can take a few minutes...
issues_with_comments_dataset = issues_dataset.map(
    lambda x: {"comments": get_comments(x["number"])}
)

Map:   0%|          | 0/6936 [00:00<?, ? examples/s]

TypeError: string indices must be integers

Uploading our dataset to hugging face hub!

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

The below code didn't work cause the above code of issues_with_comments_dataset didn't execute.

Will be resolved soon :(

In [ ]:
#uploading of the dataset
issues_with_comments_dataset.push_to_hub("github-issues")

NameError: name 'issues_with_comments_dataset' is not defined

From here, anyone can download the dataset by simply providing load_dataset() with the repository ID as the path argument.

In [ ]:
remote_dataset = load_dataset("lewtun/github-issues", split="train")
remote_dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Repo card metadata block was not found. Setting CardData to empty.


Generating train split:   0%|          | 0/3019 [00:00<?, ? examples/s]

Dataset({
    features: ['url', 'repository_url', 'labels_url', 'comments_url', 'events_url', 'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels', 'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments', 'created_at', 'updated_at', 'closed_at', 'author_association', 'active_lock_reason', 'pull_request', 'body', 'timeline_url', 'performed_via_github_app', 'is_pull_request'],
    num_rows: 3019
})